In [ ]:
from skimage.segmentation import mark_boundaries, slic
from skimage.util import img_as_float
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
import argparse
import os

In [ ]:
def cambio_tam(img, mask_m):

    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    gray = cv.GaussianBlur(gray, (7, 7), 3)
    t, dst = cv.threshold(gray, 0, 255, cv.THRESH_BINARY | cv.THRESH_TRIANGLE)
    contours, _= cv.findContours(dst, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    a = []

    for c in contours:
        area = cv.contourArea(c)
        a.append(area)
        
    for f in contours:
        area = cv.contourArea(f)
        if area == max(a):
            x, y, w, h = cv.boundingRect(f)
            imgr = img[y:y + h, x:x + w]
            mask_f1= mask_m[y:y + h, x:x + w]

    return cv.cvtColor(imgr, cv.COLOR_BGR2RGB), mask_f1

In [ ]:
def lista_E(DIR_img_ent, DIR_mask_M):
    input_img_ent= sorted(
        [
            os.path.join(DIR_img_ent, fname)
            for fname in os.listdir(DIR_img_ent)
        ]
    )

    input_img_M= sorted(
        [
            os.path.join(DIR_mask_M, fname)
            for fname in os.listdir(DIR_mask_M)
        ]
    )
    
    return input_img_ent, input_img_M

In [ ]:
def revision(img, mask):
	f= 0; i= 0; posh_0=0; posh_1= 0
	posv_1= 0; posv_0= 0
	while i < mask.shape[1]:
		if np.sum(mask[:, i])> 0 and f== 0:
			posh_0= i
			f= 1
		
		if np.sum(mask[:, i])== 0 and f== 1 or i== (mask.shape[1]-1):
			posh_1= i
			break
		i += 1

	f= 0; i= 0
	while i < mask.shape[0]:
		if np.sum(mask[i, :])> 0 and f== 0:
			posv_0= i
			f= 1
		
		if np.sum(mask[i, :])== 0 and f== 1 or i== (mask.shape[0]-1):
			posv_1= i
			break
		i += 1

	return [posv_0, posv_1, posh_0, posh_1]

In [ ]:
def super_pixel(img, n_s, sig, mostrar= False):
    segmentos= slic(img_as_float(img), n_segments= n_s, sigma= sig, start_label= 1)
    
    if mostrar== True:
        fig= plt.figure("Super pixeles")
        ax= fig.add_subplot(1, 1, 1)
        ax.imshow(mark_boundaries(img_as_float(cv.cvtColor(img, cv.COLOR_BGR2RGB)), segmentos))
        plt.axis("off")
        plt.show()
    
    return segmentos

In [ ]:
def equalize_clahe_color(img):
    cla = cv.createCLAHE(clipLimit=4.0)     
    channels = cv.split(img)     
    eq_channels = []
    for ch in channels:         
        eq_channels.append(cla.apply(ch))     
        eq_image = cv.merge(eq_channels)     
    return eq_image

def pre_procesamiento(img):
    return cv.cvtColor(equalize_clahe_color(cv.cvtColor(img, cv.COLOR_BGR2RGB)), cv.COLOR_BGR2RGB)

In [ ]:
def start_valores(img_o, img_m, SAV_o, SAV_m, s= 0):
    for i, m in zip(img_o, img_m):
        img= cv.imread(i)
        mask_m= cv.imread(m)

        img, mask_m= cambio_tam(img, mask_m)
        segmentos= super_pixel(img, 30, 20, True)
        
        for segVal in np.unique(segmentos):
            mask= np.zeros(img.shape[:2], dtype = "uint8")
            mask[segmentos== segVal]= 255
            
            if np.sum(cv.bitwise_and(img, img, mask = mask)) > 0:
                posv_0, posv_1, posh_0, posh_1= revision(img, mask)
                img_c= img[posv_0: posv_1, posh_0: posh_1]
                maskm= mask_m[posv_0: posv_1, posh_0: posh_1]
                
                if np.sum(img_c/255)> (img_c.shape[0]* img_c.shape[1])/4:
                    img_c= pre_procesamiento(img_c)
                    img_c= cv.resize(img_c,(300,300))
                    maskm= cv.resize(maskm,(300,300))
                    cv.imwrite(SAV_o+str(s)+'.png', img_c)
                    cv.imwrite(SAV_m+str(s)+'.png', maskm)
                    s += 1
                    
    return s

In [ ]:
DIR_orig_CH= "/home/revientaelp/Documentos/Bases de datos/Kika/CHASEDB1/Original"
DIR_mask_CH= "/home/revientaelp/Documentos/Bases de datos/Kika/CHASEDB1/Mask"

DIR_orig_DR= "/home/revientaelp/Documentos/Bases de datos/Kika/DRHAGIS/Fundus_Images"
DIR_mask_DR= "/home/revientaelp/Documentos/Bases de datos/Kika/DRHAGIS/Manual_Segmentations"

DIR_orig_HR_d= "/home/revientaelp/Documentos/Bases de datos/Kika/HRF_Segmentacion/diabetic_retinopathy"
DIR_mask_HR_d= "/home/revientaelp/Documentos/Bases de datos/Kika/HRF_Segmentacion/diabetic_retinopathy_manualsegm"
DIR_orig_HR_g= "/home/revientaelp/Documentos/Bases de datos/Kika/HRF_Segmentacion/glaucoma"
DIR_mask_HR_g= "/home/revientaelp/Documentos/Bases de datos/Kika/HRF_Segmentacion/glaucoma_manualsegm"
DIR_orig_HR_h= "/home/revientaelp/Documentos/Bases de datos/Kika/HRF_Segmentacion/healthy"
DIR_mask_HR_h= "/home/revientaelp/Documentos/Bases de datos/Kika/HRF_Segmentacion/healthy_manualsegm"

DIR_orig_S= "/home/revientaelp/Documentos/Bases de datos/Kika/STARE/stare-images"
DIR_mask_S= "/home/revientaelp/Documentos/Bases de datos/Kika/STARE/labels-vk"

SAV_orig= "/home/revientaelp/Documentos/Bases de datos/Kika/300/todos/org/"
SAV_mask= "/home/revientaelp/Documentos/Bases de datos/Kika/300/todos/msk/"

img_o, img_m= lista_E(DIR_orig_CH, DIR_mask_CH)
s= start_valores(img_o, img_m, SAV_orig, SAV_mask)

img_o, img_m= lista_E(DIR_orig_DR, DIR_mask_DR)
s= start_valores(img_o, img_m, SAV_orig, SAV_mask)

img_o, img_m= lista_E(DIR_orig_HR_d, DIR_mask_HR_d)
s= start_valores(img_o, img_m, SAV_orig, SAV_mask)
img_o, img_m= lista_E(DIR_orig_HR_g, DIR_mask_HR_g)
s= start_valores(img_o, img_m, SAV_orig, SAV_mask)
img_o, img_m= lista_E(DIR_orig_HR_h, DIR_mask_HR_h)
s= start_valores(img_o, img_m, SAV_orig, SAV_mask)

img_o, img_m= lista_E(DIR_orig_S, DIR_mask_S)
s= start_valores(img_o, img_m, SAV_orig, SAV_mask)

In [15]:
import numpy as np
import random
import cv2
import os

In [16]:
def horizontal_flip(img, flag= True):
    if flag:
        return cv2.flip(img, 1)
    else:
        return img

def vertical_flip(img, flag= True):
    if flag:
        return cv2.flip(img, 0)
    else:
        return img

def rotacion(img, r=1):
    largo, ancho= img.shape[:2]
    M= cv2.getRotationMatrix2D((ancho/2, largo/2), 270/r, 1)
    dst_image= cv2.warpAffine(img, M, (ancho, largo))
    return dst_image

In [17]:
def lista_E(DIR_sin, DIR_con):
    input_img_0= sorted(
        [
            os.path.join(DIR_sin, fname)
            for fname in os.listdir(DIR_sin)
        ]
    )

    input_img_1= sorted(
        [
            os.path.join(DIR_con, fname)
            for fname in os.listdir(DIR_con)
        ]
    )

    return input_img_0, input_img_1

In [18]:
DIR_org= "/home/revientaelp/Documentos/Bases de datos/Kika/300/todos/org"
DIR_msk= "/home/revientaelp/Documentos/Bases de datos/Kika/300/todos/msk"

SAV_org= "/home/revientaelp/Documentos/Bases de datos/Kika/300/Aumento/Org/"
SAV_msk= "/home/revientaelp/Documentos/Bases de datos/Kika/300/Aumento/Msk/"


ii0, ii1= lista_E(DIR_org, DIR_msk)

f= 0
for t, i in zip(ii0, ii1):
    img= cv2.imread(t)
    msk= cv2.imread(i)

    cv2.imwrite(SAV_org+str(f)+'.png', img)
    cv2.imwrite(SAV_msk+str(f)+'.png', msk)
    
    
    cv2.imwrite(SAV_org+str(f+1)+'.png', horizontal_flip(img))
    cv2.imwrite(SAV_msk+str(f+1)+'.png', horizontal_flip(msk))    
    
    cv2.imwrite(SAV_org+str(f+2)+'.png', vertical_flip(img))
    cv2.imwrite(SAV_msk+str(f+2)+'.png', vertical_flip(msk))    
    
    cv2.imwrite(SAV_org+str(f+3)+'.png', horizontal_flip(vertical_flip(img)))
    cv2.imwrite(SAV_msk+str(f+3)+'.png', horizontal_flip(vertical_flip(msk)))
    
    
    r= (np.random.randint(15)+1)
    cv2.imwrite(SAV_org+str(f+4)+'.png', rotacion(vertical_flip(img), r))
    cv2.imwrite(SAV_msk+str(f+4)+'.png', rotacion(vertical_flip(msk), r))
    
    
    r= (np.random.randint(15)+1)
    cv2.imwrite(SAV_org+str(f+5)+'.png', rotacion(horizontal_flip(img), r))
    cv2.imwrite(SAV_msk+str(f+5)+'.png', rotacion(horizontal_flip(msk), r))
    f += 6

In [21]:
import numpy as np
import cv2
import os

#Paths de donde salen las imagenes
img_path_o= "/home/revientaelp/Documentos/Bases de datos/Kika/300/Aumento/Org/"
img_path_m= "/home/revientaelp/Documentos/Bases de datos/Kika/300/Aumento/Msk/"

#Paths de donde se guardaran los aumentos de datos
sav_path_ent_o= "/home/revientaelp/Documentos/Bases de datos/Kika/300/ent/org/"
sav_path_ent_m= "/home/revientaelp/Documentos/Bases de datos/Kika/300/ent/msk/"

sav_path_val_o= "/home/revientaelp/Documentos/Bases de datos/Kika/300/val/org/"
sav_path_val_m= "/home/revientaelp/Documentos/Bases de datos/Kika/300/val/msk/"

input_img_paths = sorted(
    [
        os.path.join(img_path_o, fname)
        for fname in os.listdir(img_path_o) 
        if fname.endswith(".png")
    ]
)

lista= np.arange(len(input_img_paths))
np.random.shuffle(lista)

i= 0; z= 0
for j in lista:
    img= cv2.imread(img_path_o+str(j)+'.png')
    msk= cv2.imread(img_path_m+str(j)+'.png')

    if i < int(len(lista)*.9):
        cv2.imwrite(sav_path_ent_o+str(i)+'.png', img)
        cv2.imwrite(sav_path_ent_m+str(i)+'.png', msk)
        i += 1
    else:
        cv2.imwrite(sav_path_val_o+str(z)+'.png', img)
        cv2.imwrite(sav_path_val_m+str(z)+'.png', msk)
        z += 1
